References:
* https://www.kaggle.com/cdeotte/triple-stratified-kfold-with-tfrecords
* https://www.kaggle.com/dimitreoliveira/cassava-leaf-disease-tpu-tensorflow-inference/data
* https://www.kaggle.com/dimitreoliveira/cassava-leaf-disease-tpu-tensorflow-training/notebook#Training

Training Notebook:
https://www.kaggle.com/ajaykumar7778/tpu-train-effnets/output?scriptVersionId=48358834

In [ ]:
!pip install --quiet /kaggle/input/kerasapplications
!pip install --quiet /kaggle/input/efficientnet-git

In [ ]:
import math, os, re, warnings, random
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from kaggle_datasets import KaggleDatasets
from sklearn.utils import class_weight
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
from tensorflow.keras import optimizers, applications, Sequential, losses, metrics
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
import efficientnet.tfkeras as efn

def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed = 0
seed_everything(seed)
warnings.filterwarnings('ignore')

In [ ]:
# TPU or GPU detection
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print(f'Running on TPU {tpu.master()}')
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')



In [ ]:
DEVICE = "GPU" #or "TPU"

# train = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
# test  = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
sub   = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')

FOLDS = 3
# WHICH IMAGE SIZES TO LOAD EACH FOLD
# CHOOSE 128, 192, 256, 384, 512, 768
IMG_SIZES = [256, 384, 512]
# INCLUDE OLD COMP DATA? YES=1 NO=0
# INC2019 = [1,0,1,1,0,1,0,1]
# INC2018 = [0,1,1,0,1,1,1,1]
BATCH_SIZES = [64,64,64]   #[64,64,64]
EPOCHS = [20, 20, 20]   #[10, 10, 10]
# WHICH EFFICIENTNET B? TO USE
EFF_NETS = [2, 2, 3]
# WEIGHTS FOR FOLD MODELS WHEN PREDICTING TEST
WGTS = [1/FOLDS]*FOLDS#[1/FOLDS,1/FOLDS,1/FOLDS]
# TEST TIME AUGMENTATION STEPS
TTA_STEPS = 3

In [ ]:
BATCH_SIZE = 16 * REPLICAS
HEIGHT = 512
WIDTH = 512
CHANNELS = 3
N_CLASSES = 5

In [ ]:
EFNS = [efn.EfficientNetB0, efn.EfficientNetB1, efn.EfficientNetB2, efn.EfficientNetB3, 
        efn.EfficientNetB4, efn.EfficientNetB5, efn.EfficientNetB6]

def build_model(dim=128, ef=0):
    input_image = tf.keras.layers.Input(shape=(dim,dim,3))
    #base = EFNS[ef](input_shape=(dim,dim,3),weights='imagenet',include_top=False)
    
    base_model = EFNS[ef](input_tensor=input_image, 
                                    include_top=False, 
                                    #weights='noisy-student', 
                                    weights = None,
                                    pooling='avg')
    
    model = Sequential([
                base_model,
                L.Dropout(.25),
                L.Dense(N_CLASSES, activation='softmax', name='output')
            ])
    
#     optimizer = optimizers.Adam(lr=LEARNING_RATE)
#     model.compile(optimizer=optimizer, 
#                   loss=losses.SparseCategoricalCrossentropy(), 
#                   metrics=['sparse_categorical_accuracy'])
    
    return model

In [ ]:
def count_data_items(filenames):
    n = [int(re.compile(r'-([0-9]*)\.').search(filename).group(1)) for filename in filenames]
    return np.sum(n)


database_base_path = '/kaggle/input/cassava-leaf-disease-classification/'

CLASSES = ['Cassava Bacterial Blight', 
           'Cassava Brown Streak Disease', 
           'Cassava Green Mottle', 
           'Cassava Mosaic Disease', 
           'Healthy']


In [ ]:
database_base_path = '/kaggle/input/cassava-leaf-disease-classification/'
submission = pd.read_csv(f'{database_base_path}sample_submission.csv')
display(submission.head())

TEST_FILENAMES = tf.io.gfile.glob(f'{database_base_path}test_tfrecords/ld_test*.tfrec')
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
print(f'GCS: test: {NUM_TEST_IMAGES}')

In [ ]:
import glob

model_path_list = glob.glob('../input/exp-1-3-fold/fold*.h5')
model_path_list.sort()


print('Models to predict:')
print(*model_path_list, sep='\n')

In [ ]:
# model_path.split('/')[-1].split('.')[0].split('-')[-1]

In [ ]:
def data_augment(image, label):
    p_spatial = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_crop = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    
    # Flips
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    if p_spatial > .75:
        image = tf.image.transpose(image)
    # Rotates
    if p_rotate > .75:
        image = tf.image.rot90(image, k=3) # rotate 270º
    elif p_rotate > .5:
        image = tf.image.rot90(image, k=2) # rotate 180º
    elif p_rotate > .25:
        image = tf.image.rot90(image, k=1) # rotate 90º
        
    return image, label

In [ ]:
# Datasets utility functions
def get_name(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    name = parts[-1]
    return name

def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    
#     image = center_crop(image)
    return image

def center_crop(image, h, w):
    image = tf.reshape(image, [600, 800, CHANNELS]) # Original shape
    
    h, w = image.shape[0], image.shape[1]
    if h > w:
        image = tf.image.crop_to_bounding_box(image, (h - w) // 2, 0, w, w)
    else:
        image = tf.image.crop_to_bounding_box(image, 0, (w - h) // 2, h, h)
        
    image = tf.image.resize(image, [h, w]) # Expected shape
    return image

def resize_image(image, label, h, w):
    image = tf.image.resize(image, [h, w])
    image = tf.reshape(image, [h, w, CHANNELS])
    return image, label

def process_path(file_path):
    name = get_name(file_path)
    img = tf.io.read_file(file_path)
    img = decode_image(img)
    return img, name

def get_dataset(files_path, h, w, shuffled=False, tta=False, extension='jpg'):
    dataset = tf.data.Dataset.list_files(f'{files_path}*{extension}', shuffle=shuffled)
    dataset = dataset.map(process_path, num_parallel_calls=AUTO)
    if tta:
        dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    #dataset = dataset.map(resize_image, num_parallel_calls=AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

In [ ]:


files_path = f'{database_base_path}/test_images/'
test_preds = np.zeros((len(os.listdir(files_path)), N_CLASSES))


for model_path in model_path_list:
    fold = int(model_path.split('/')[-1].split('.')[0].split('-')[-1])
    print(model_path)
    h = IMG_SIZES[fold]
    w = IMG_SIZES[fold]
#     K.clear_session()
#     model.load_weights(model_path)

    K.clear_session()
    with strategy.scope():
        model = build_model(dim=IMG_SIZES[fold],ef=EFF_NETS[fold])
    
    print('Loading model...')
    model.load_weights(model_path)
    #model.load_weights('../input/exp-1-3-fold/fold-%i.h5'%fold)

    if TTA_STEPS > 0:
        test_ds = get_dataset(files_path, h, w, tta=True)
        for step in range(TTA_STEPS):
            print(f'TTA step {step+1}/{TTA_STEPS}')
            x_test = test_ds.map(lambda image, image_name: image)
            test_preds += model.predict(x_test) / (TTA_STEPS * len(model_path_list))
    else:
        test_ds = get_dataset(files_path, h, w, tta=False)
        x_test = test_ds.map(lambda image, image_name: image)
        test_preds += model.predict(x_test) / len(model_path_list)
    
test_preds = np.argmax(test_preds, axis=-1)
image_names = [img_name.numpy().decode('utf-8') for img, img_name in iter(test_ds.unbatch())]



In [ ]:
submission = pd.DataFrame({'image_id': image_names, 'label': test_preds})
submission.to_csv('submission.csv', index=False)
display(submission.head())

In [ ]:
# for fold, history in enumerate(history_list):
#     print(f'\nFOLD: {fold+1}')
#     plot_metrics(history)

Model evaluation

Now we can evaluate the performance of the model, first, we can evaluate the usual metrics like, accuracy, precision, recall, and f1-score, scikit-learn provides the perfect function for this classification_report.

We are evaluating the model on the OOF predictions, it stands for Out Of Fold, since we are training using K-Fold our model will see all the data, and the correct way to evaluate each fold is by looking at the predictions that are not from that fold.
OOF metrics

In [ ]:
# y_true = np.concatenate(oof_labels)
# y_preds = np.concatenate(oof_pred)

# print(classification_report(y_true, y_preds, target_names=CLASSES))


Confusion matrix

Let's also take a look at the confusion matrix, this will give us an idea about what classes the model is mixing or having a hard time.


In [ ]:


# fig, ax = plt.subplots(1, 1, figsize=(20, 12))
# train_cfn_matrix = confusion_matrix(y_true, y_preds, labels=range(len(CLASSES)))
# train_cfn_matrix = (train_cfn_matrix.T / train_cfn_matrix.sum(axis=1)).T
# train_df_cm = pd.DataFrame(train_cfn_matrix, index=CLASSES, columns=CLASSES)
# ax = sns.heatmap(train_df_cm, cmap='Blues', annot=True, fmt='.2f', linewidths=.5).set_title('Train', fontsize=30)
# plt.show()



Visualize predictions

Finally, it is a good practice to always inspect some of the model's prediction by looking at the data, this can give an idea if the model is getting some predictions wrong because the data is really hard, of if it is because the model is actually bad.
Class map

0: Cassava Bacterial Blight (CBB)
1: Cassava Brown Streak Disease (CBSD)
2: Cassava Green Mottle (CGM)
3: Cassava Mosaic Disease (CMD)
4: Healthy

In [ ]:
# train_dataset = get_dataset(TRAINING_FILENAMES, ordered=True)

In [ ]:
# x_samp, y_samp = dataset_to_numpy_util(train_dataset, 25)

# x_samp_1, y_samp_1 = x_samp[:9,:,:,:], y_samp[:9]
# samp_preds_1 = model.predict(x_samp_1, batch_size=9)
# display_9_images_with_predictions(x_samp_1, samp_preds_1, y_samp_1)

# x_samp_2, y_samp_2 = x_samp[9:,:,:,:], y_samp[9:]
# samp_preds_2 = model.predict(x_samp_2, batch_size=9)
# display_9_images_with_predictions(x_samp_2, samp_preds_2, y_samp_2)